# 2.0 — Panorama Geral dos dados (Kaggle) e Pré-Limpeza

**Objetivo:** carregar `kaggle_movies_metadata.csv`, analisar a estrutura do dataset (dimensão, tipos de dados, valores nulos) e inspecionar os campos críticos para o match (título, título original, data de lançamento/ano, votos, média de votos, orçamento, receita, runtime, idioma).

Em seguida, aplicar uma **pré-limpeza mínima** nessas colunas-chave, incluindo: padronização de ano, ajuste de duração, normalização de valores de receita/orçamento e uniformização dos títulos, suficiente para viabilizar o matching com a base Lighthouse no notebook 03.  

**Saída:** **Saída:** Arquivo limpo e padronizado em `data/processed/kaggle_movies_metadata_clean.csv`

## 2.1 - Imports

In [1]:
# =====================================================
# Imports
# =====================================================

# Manipulação e análise de dados
import math
import numpy as np
import pandas as pd
import re
from pathlib import Path

# Visualização de dados
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

# Sistema e paths
import os
from pathlib import Path

# Utilidades para notebooks
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML

# Manipulação de datas
import datetime

## 2.2 - Funções Auxiliares

In [2]:
# ==== Caminhos & Display ====
class PATHS:
    """
    Caminhos padrão do projeto quando o notebook roda dentro de 'notebooks/'.

    Regra:
    - Se o cwd termina com 'notebooks', a raiz é o pai (../)
    - Caso contrário, assume que já estamos na raiz (útil se abrir o Jupyter na raiz)
    """
    _CWD = Path.cwd()
    ROOT = _CWD.parent if _CWD.name == "notebooks" else _CWD
    RAW  = ROOT / "data" / "raw"
    PROC = ROOT / "data" / "processed"
    INTER = ROOT / "data" / "intermediary"
    REP  = ROOT / "reports"
    FIG  = REP / "figures"

def ensure_dirs():
    """Garante que as pastas principais existam."""
    for p in [PATHS.RAW, PATHS.INTER, PATHS.PROC, PATHS.REP, PATHS.FIG]:
        p.mkdir(parents=True, exist_ok=True)

def set_display(max_cols: int = 100, decimals: int = 2):
    """Ajusta visualização padrão no Pandas/Seaborn para leitura eficiente."""
    pd.options.display.max_columns = max_cols
    fmt = "{:." + str(decimals) + "f}"
    pd.options.display.float_format = fmt.format
    sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# ==== Parsing & Normalização ====
def normalize_title(s):
    """Normaliza título: minúsculas, sem pontuação, sem espaços extras."""
    if pd.isna(s):
        return np.nan
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s.strip()

def parse_year(x):
    """Extrai o primeiro ano encontrado (ex.: '1994' ou '1994(I)')."""
    if pd.isna(x):
        return pd.NA
    m = re.search(r"\d{4}", str(x))
    return int(m.group()) if m else pd.NA

def to_numeric_safe(series):
    """Converte para numérico com 'coerce' (incompatíveis viram NaN)."""
    return pd.to_numeric(series, errors="coerce")

# ==== Deduplicação ====
def dedup_by_key(df, keys, score_col="vote_count"):
    """
    Se houver múltiplas linhas para a mesma chave, mantém a de maior `score_col`.
    Ex.: ao agrupar por (title_norm, Year), fica a linha com maior número de votos.
    """
    tmp = df.copy()
    tmp[score_col] = to_numeric_safe(tmp[score_col]) if score_col in tmp.columns else 0
    tmp = tmp.sort_values(by=[score_col], ascending=False)
    return tmp.drop_duplicates(subset=keys, keep="first")

In [3]:
# Chamo as de caminhos e display
ensure_dirs()
set_display()

## 2.3 - Carregando os Dados

In [4]:
# Realizo a leitura dos dados através da classe
df_km = pd.read_csv(PATHS.RAW / "kaggle_movies_metadata.csv", low_memory=False)

## 2.4 - Descrição dos Dados

In [5]:
# Cópia da base - Boa prática para evitar modificações sem intenção
df1 = df_km.copy()

In [6]:
print("Kaggle Metabase:", df1.shape)

Kaggle Metabase: (45466, 24)


In [7]:
print("Kaggle Metabase:", df1.columns)

Kaggle Metabase: Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')


In [8]:
print("\nTipos de dados da base Kaggle Metabase:\n", df1.dtypes)


Tipos de dados da base Kaggle Metabase:
 adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object


In [9]:
display(df1.sample(3, random_state=42))

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
43526,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",https://www.netflix.com/title/80164212,411405,tt5717492,en,Small Crimes,"A disgraced former cop, fresh off a six-year p...",7.219022,/z7jmLmrs0pLlDU4GI6ItaJeqlET.jpg,"[{'name': 'Rooks Nest Entertainment', 'id': 34...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",2017-04-28,0.00,95.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Small Crimes,False,5.80,55.00
6383,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",NaN,42492,tt0069449,en,Up the Sandbox,"A young wife and mother, bored with day-to-day...",0.13845,/zwOmdqvPObv9EsMgne7EBYzPYGW.jpg,"[{'name': 'Barwood Films', 'id': 3645}, {'name...","[{'iso_3166_1': 'US', 'name': 'United States o...",1972-12-21,0.00,97.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Up the Sandbox,False,7.30,2.00
3154,False,NaN,1000000,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,12143,tt0103759,en,Bad Lieutenant,"While investigating a young nun's rape, a corr...",6.417037,/oe8VjWCKXktqA19T1ZWtaSn8rc2.jpg,"[{'name': 'Bad Lt. Productions', 'id': 11264}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1992-09-16,2019469.00,96.00,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Gambler. Thief. Junkie. Killer. Cop.,Bad Lieutenant,False,6.90,162.00


In [10]:
display(df1.isna().mean().sort_values(ascending=False)*100)

belongs_to_collection   90.12
homepage                82.88
tagline                 55.10
overview                 2.10
poster_path              0.85
runtime                  0.58
status                   0.19
release_date             0.19
imdb_id                  0.04
original_language        0.02
vote_average             0.01
vote_count               0.01
title                    0.01
video                    0.01
spoken_languages         0.01
revenue                  0.01
popularity               0.01
production_countries     0.01
production_companies     0.01
genres                   0.00
id                       0.00
adult                    0.00
budget                   0.00
original_title           0.00
dtype: float64

In [11]:
df1.nunique(dropna=True)

adult                        5
belongs_to_collection     1698
budget                    1226
genres                    4069
homepage                  7673
id                       45436
imdb_id                  45417
original_language           92
original_title           43373
overview                 44307
popularity               43758
poster_path              45024
production_companies     22708
production_countries      2393
release_date             17336
revenue                   6863
runtime                    353
spoken_languages          1931
status                       6
tagline                  20283
title                    42277
video                        2
vote_average                92
vote_count                1820
dtype: int64

## 2.5 - Pré-limpeza dos dados da Kaggle Movies (para matching)

A partir do diagnóstico por coluna, vou padronizar apenas os campos necessários para o matching:

- Ano: extrairei de "release_date" uma coluna Year numérica.

- Títulos normalizados: criarei um "title_norm" (a partir de title) e "original_title_norm" (a partir de original_title) em minúsculas, sem pontuação e com espaços normalizados — isso reduzirá variações superficiais em joins.

- Numéricos confiáveis: garantirei float/int em "budget", "revenue", "runtime", "vote_average", "vote_count" e "popularity" via conversão com coerce (valores inválidos viram NaN).

Não vou renomear as colunas originais agora para preservar a compatibilidade, nem irei imputar valores ausentes/outliers (fica para a EDA). A saída aqui será um CSV “limpo o suficiente” para o matching no próximo notebook.


In [12]:
df_clean = df1.copy()

In [13]:
# Crio a coluna Year e normalizo os títulos
df_clean["Year"] = df_clean["release_date"].map(parse_year).astype("Int64")
df_clean["title_norm"] = df_clean["title"].map(normalize_title)
df_clean["original_title_norm"] = df_clean["original_title"].map(normalize_title)

In [14]:
# Faço as conversões numéricas “seguras” (coerce: erros viram NaN) das colunas abaixo
for col in ["budget", "revenue", "runtime", "vote_average", "vote_count", "popularity"]:
    df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

In [15]:
# Faço as checagens rápidas pós limpeza
display("Tipos (pós limpeza, colunas-chave):", df_clean[["title","original_title","Year","budget","revenue","runtime","vote_average","vote_count","popularity"]].dtypes)

cols_key = ["title","original_title","Year","budget","revenue","runtime","vote_average","vote_count","popularity"]
display("Porcentagem de nulos (pós limpeza, colunas-chave):", df_clean[cols_key].isna().mean().sort_values(ascending=False) * 100)

'Tipos (pós limpeza, colunas-chave):'

title              object
original_title     object
Year                Int64
budget            float64
revenue           float64
runtime           float64
vote_average      float64
vote_count        float64
popularity        float64
dtype: object

'Porcentagem de nulos (pós limpeza, colunas-chave):'

runtime          0.58
Year             0.20
title            0.01
vote_average     0.01
revenue          0.01
vote_count       0.01
popularity       0.01
budget           0.01
original_title   0.00
dtype: float64

In [16]:
# Testo as duplicatas potenciais para as chaves de match
dup1 = df_clean.duplicated(subset=["title_norm","Year"], keep=False).sum()
dup2 = df_clean.duplicated(subset=["original_title_norm","Year"], keep=False).sum()
print(f"Duplicatas por (title_norm, Year): {dup1}")
print(f"Duplicatas por (original_title_norm, Year): {dup2}")

Duplicatas por (title_norm, Year): 174
Duplicatas por (original_title_norm, Year): 109


In [17]:
display(df_clean.loc[:, ["title","original_title","Year","title_norm","original_title_norm","vote_average","vote_count"]]
        .sample(5, random_state=42))

,title,original_title,Year,title_norm,original_title_norm,vote_average,vote_count
43526,Small Crimes,Small Crimes,2017,small crimes,small crimes,5.80,55.00
6383,Up the Sandbox,Up the Sandbox,1972,up the sandbox,up the sandbox,7.30,2.00
3154,Bad Lieutenant,Bad Lieutenant,1992,bad lieutenant,bad lieutenant,6.90,162.00
10146,Satan's Little Helper,Satan's Little Helper,2004,satans little helper,satans little helper,5.00,42.00
9531,Sitcom,Sitcom,1998,sitcom,sitcom,6.40,27.00


## 2.6 - Deduplicação por chave - registro por mais votos

In [18]:
# Chave 1: (title_norm, Year)
km_key1 = dedup_by_key(df_clean, keys=["title_norm","Year"], score_col="vote_count")

# Chave 2 (fallback): (original_title_norm, Year)
km_key2 = dedup_by_key(df_clean, keys=["original_title_norm","Year"], score_col="vote_count")

print("Tamanhos após dedup:")
print("km_key1:", km_key1.shape, "| km_key2:", km_key2.shape)

# Chegando
display(km_key1[["title","Year","vote_count","vote_average"]].head(5))
display(km_key2[["original_title","Year","vote_count","vote_average"]].head(5))


Tamanhos após dedup:
km_key1: (45376, 27) | km_key2: (45411, 27)


,title,Year,vote_count,vote_average
15480,Inception,2010,14075.00,8.10
12481,The Dark Knight,2008,12269.00,8.30
14551,Avatar,2009,12114.00,7.20
17818,The Avengers,2012,12000.00,7.40
26564,Deadpool,2016,11444.00,7.40


,original_title,Year,vote_count,vote_average
15480,Inception,2010,14075.00,8.10
12481,The Dark Knight,2008,12269.00,8.30
14551,Avatar,2009,12114.00,7.20
17818,The Avengers,2012,12000.00,7.40
26564,Deadpool,2016,11444.00,7.40


## 2.7 - Salvando os datasets limpos e deduplicados

In [19]:
# Salvo os datasets limpos e as chaves de match
out_key1  = PATHS.INTER / "kaggle_movies_key_title_year.csv"
out_key2  = PATHS.INTER / "kaggle_movies_key_originaltitle_year.csv"

km_key1.to_csv(out_key1, index=False)
km_key2.to_csv(out_key2, index=False)

print("Arquivos salvos:")
print("Key1:", out_key1.resolve())
print("Key2:", out_key2.resolve())


Arquivos salvos:
Key1: /home/emersds/repos_projetos/project_lighthouse/data/intermediary/kaggle_movies_key_title_year.csv
Key2: /home/emersds/repos_projetos/project_lighthouse/data/intermediary/kaggle_movies_key_originaltitle_year.csv


# 2.8 - Decisões e Próximos passos

**Decisões nesta etapa**
- Mantive somente as colunas necessárias para o match e enriquecimento com a base Lighthouse.
- Criei Year via função parsing de release_date e forcei Int64 para aceitar ausências sem cair em object.
- Normalizei title e original_title para chaves de junção e converti os campos numéricos com coerce.
- Preparei duas versões deduplicadas por (title_norm, Year) e (original_title_norm, Year) mantendo o registro com maior vote_count.
- Salvei as saídas limpas em data/processed/ para consumo direto no notebook de matching.

**Próximos passos**
- Carregar lighthouse_clean.csv, kaggle_movies_key_title_year.csv e kaggle_movies_key_originaltitle_year.csv.
- Fazer o match 1 (title_norm+Year) e fallback 2 (original_title_norm+Year).
- Enriquecer a base Lighthouse com budget, revenue, runtime, vote_*, popularity e salvar lighthouse_enriched.csv.